In [2]:
from prefect.blocks.notifications import CustomWebhookNotificationBlock

custom_webhook_block = CustomWebhookNotificationBlock.load("feishu")

custom_webhook_block.notify("Hello from Prefect!")

AttributeError: 'coroutine' object has no attribute 'notify'